# GMC - Solid Electrolyte Interphase 
Simple example of [solid-electrolyte interphase formation and evolution in a lithium-ion battery](./https://espottesmith.github.io/files/papers/spottesmith_kam_towards_mechanistic_2022.pdf)

In [1]:
import sqlite3
from itertools import product, combinations
import copy
import os

import numpy as np
from scipy.constants import pi, epsilon_0, elementary_charge

from monty.serialization import loadfn

from pymatgen.core.structure import Molecule
from pymatgen.analysis.graphs import MoleculeGraph
from pymatgen.analysis.local_env import OpenBabelNN, metal_edge_extender

In [2]:
# Room temperature (25 C) in Kelvin
ROOM_TEMP = 298.15
# ROOM_TEMP = 423.15

# Boltzmann constant in eV / K
KB = 8.617333262 * 10 ** -5

# Planck constant in eV * s
PLANCK = 4.135667696 * 10 ** -15

# Simulation variables
electron_free_energy = -1.4
dielectric = 18.5
refractive = 1.415
radius = 5.0
decay_constant=1.2

# Residence time of Li+ with EC, from Tingzheng, is roughly 5 ns (0.266 eV barrier)
min_hopping_barrier = 0.266
# min_hopping_barrier = 0.325

# kMC parameters
factor_zero = 1.0
factor_two = 1.0
factor_duplicate = 0.5

In [4]:
def eyring(dg_barrier, kappa=1.0, temperature=ROOM_TEMP):
    if dg_barrier <= 0:
        return kappa * KB * temperature / PLANCK
    else:
        return kappa * KB * temperature / PLANCK * np.exp(-dg_barrier / (KB * temperature))


def lambda_inner(a_a, b_b, a_b, b_a):
    return ((a_b - a_a) + (b_a - b_b)) / 2 * 27.2114


def lambda_outer(r=radius, d=radius, eps=dielectric, n=refractive):
    lambda_o = abs(elementary_charge) / (8 * pi * epsilon_0)
    lambda_o *= (1 / r - 1 / (2 * d)) * 10 ** 10
    lambda_o *= 1 / n ** 2 - 1 / eps
    return lambda_o


def barrier_redox(dg, a_a, b_b, a_b, b_a, dq=1, e_free=electron_free_energy, r=radius, d=radius, eps=dielectric, n=refractive):
    lambda_i = lambda_inner(a_a, b_b, a_b, b_a)
    lambda_o = lambda_outer(r=r, d=d, eps=eps, n=n)
    lambda_total = lambda_i + lambda_o

    delta_g = dg + dq * e_free

    dg_barrier = lambda_total / 4 * (1 + delta_g / lambda_total) ** 2

    return dg_barrier


def get_thermo(reaction, mols, mapping, ts):
    t = ts[reaction["ts"]]["free_energy"]
    reactants = sum([mols[mapping[r]]["free_energy"] for r in reaction["reactants"]])
    products = sum([mols[mapping[r]]["free_energy"] for r in reaction["products"]])

    return {"dg": products - reactants,
            "forward_barrier": t - reactants,
            "reverse_barrier": t - products}

In [5]:
create_metadata_table = """
    CREATE TABLE metadata (
            number_of_species   INTEGER NOT NULL,
            number_of_reactions INTEGER NOT NULL
    );
"""

insert_metadata = """
    INSERT INTO metadata VALUES (?, ?)
"""

# it is important that reaction_id is the primary key
# otherwise the network loader will be extremely slow.
create_reactions_table = """
    CREATE TABLE reactions (
            reaction_id         INTEGER NOT NULL PRIMARY KEY,
            number_of_reactants INTEGER NOT NULL,
            number_of_products  INTEGER NOT NULL,
            reactant_1          INTEGER NOT NULL,
            reactant_2          INTEGER NOT NULL,
            product_1           INTEGER NOT NULL,
            product_2           INTEGER NOT NULL,
            rate                REAL NOT NULL,
            dG                  REAL NOT NULL,
            dG_barrier          REAL NOT NULL,
            is_redox            INTEGER NOT NULL
    );
"""

insert_reaction = """
    INSERT INTO reactions VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

create_initial_state_table = """
    CREATE TABLE initial_state (
            species_id             INTEGER NOT NULL PRIMARY KEY,
            count                  INTEGER NOT NULL
    );
"""

create_trajectories_table = """
    CREATE TABLE trajectories (
            seed         INTEGER NOT NULL,
            step         INTEGER NOT NULL,
            reaction_id  INTEGER NOT NULL,
            time         REAL NOT NULL
    );
"""

create_factors_table = """
    CREATE TABLE factors (
            factor_zero      REAL NOT NULL,
            factor_two       REAL NOT NULL,
            factor_duplicate REAL NOT NULL
    );
"""

create_interrupt_state_table = """
    CREATE TABLE interrupt_state (
      seed                    INTEGER NOT NULL,
      species_id              INTEGER NOT NULL,
      count                   INTEGER NOT NULL
);
"""

create_interrupt_cutoff_table = """
    CREATE TABLE interrupt_cutoff (
      seed                    INTEGER NOT NULL,
      step                    INTEGER NOT NULL,
      time                    INTEGER NOT NULL       
);
"""

In [14]:
marcus_sp = loadfn("20211130_marcus_sp.json")
marcus_mols = [Molecule.from_dict(e["orig"]["molecule"]) for e in marcus_sp]
marcus_mgs = [metal_edge_extender(MoleculeGraph.with_local_env_strategy(m, OpenBabelNN())) for m in marcus_mols]

mols = loadfn("20211130_mol_entries.json")
mapping = {m["name"]: i for i, m in enumerate(mols)}

ec_ind = mapping["EC"]
li_plus_ind = mapping["Li_plus"]
oh_minus_ind = mapping["OH_minus"]
h_ind = mapping["H"]
co2_ind = mapping["CO2"]
liec_ind = mapping["LiEC_plus"]
ledc_ind = mapping["LEDC"]
lemc_ind = mapping["LEMC"]
carbonate_ind = mapping["Li2CO3"]
oxalate_ind = mapping["Li_oxalate"]

ts_normal = loadfn("20211130_ts.json")
ts_lowlot = loadfn("20211130_ts_lowlot.json")

base_dir = "/Users/ewcss/data/kmc/test/for_rnmc_paper"
# os.mkdir(base_dir)

forbidden_reactions = [(30, 24, 8, -1),
                       (23, -1, 24, 31)]

RuntimeError: ERROR: Openbabel must be installed in order to use Q-Chem Custodian!